* load book data
* time same col, two different aggs
* time same col, aggs as a single operation
* time same col, aggs via namedagg

* numba on wap?

In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
#from pprint import pprint
import numpy as np

import random
#import memory_profiler
#import altair as alt
#from tqdm import tqdm
import datetime

#import ipython_memory_usage
#%ipython_memory_usage_start

from utility import ROOT, TEST_CSV, TRAIN_CSV
#df_train_all = pd.read_csv(TRAIN_CSV)
#df_train_all = df_train_all.set_index(['stock_id', 'time_id'])

In [40]:
filename = 'book_train.parquet'
stock_id = 1
assert isinstance(stock_id, int)
df_book_train_stock_X = pd.read_parquet(
    os.path.join(ROOT, f"{filename}/stock_id={stock_id}")
)
df_book_train_stock_X["stock_id"] = stock_id
df_book_train_stock_X = df_book_train_stock_X.set_index(['stock_id', 'time_id'])
display(df_book_train_stock_X.shape)
df_book_train_stock_X.reset_index().time_id.unique()

(1507532, 9)

array([    5,    11,    16, ..., 32758, 32763, 32767])

In [88]:
df_book_train_stock_X.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1507532 entries, 0 to 1507531
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   stock_id           1507532 non-null  int64  
 1   time_id            1507532 non-null  int64  
 2   seconds_in_bucket  1507532 non-null  int16  
 3   bid_price1         1507532 non-null  float32
 4   ask_price1         1507532 non-null  float32
 5   bid_price2         1507532 non-null  float32
 6   ask_price2         1507532 non-null  float32
 7   bid_size1          1507532 non-null  int32  
 8   ask_size1          1507532 non-null  int32  
 9   bid_size2          1507532 non-null  int32  
 10  ask_size2          1507532 non-null  int32  
dtypes: float32(4), int16(1), int32(4), int64(2)
memory usage: 71.9 MB


In [42]:
%timeit df_book_train_stock_X.groupby('time_id').mean()

107 ms ± 866 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
%timeit df_book_train_stock_X.groupby('time_id').agg('mean')

106 ms ± 218 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [49]:
%timeit df_book_train_stock_X.groupby('time_id').agg('median')

239 ms ± 7.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%timeit df_book_train_stock_X.groupby('time_id').agg('max')

84.5 ms ± 443 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
%timeit df_book_train_stock_X.groupby('time_id').agg('count')

60.6 ms ± 565 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
%timeit df_book_train_stock_X.groupby('time_id').agg('size')

31.5 ms ± 773 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [53]:
%%timeit
res1 = df_book_train_stock_X.groupby('time_id').agg('mean')
res2 = df_book_train_stock_X.groupby('time_id').agg('median')
pd.concat((res1, res2), axis=1)

345 ms ± 9.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%timeit df_book_train_stock_X.groupby('time_id').agg(['mean', 'median'])

435 ms ± 27.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit df_book_train_stock_X.groupby('time_id').agg(['mean', 'median'])

429 ms ± 754 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
df_book_train_stock_X_noidx = df_book_train_stock_X.reset_index()
df_book_train_stock_X_noidx.head(2)

,stock_id,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,1,5,0,1.000754,1.001542,1.000689,1.001607,1,25,25,100
1,1,5,1,1.000754,1.001673,1.000689,1.001739,26,60,25,100


In [60]:
%timeit df_book_train_stock_X_noidx.groupby('time_id').mean()

158 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
df_book_train_stock_X_noidx['time_id_cat'] = df_book_train_stock_X_noidx['time_id'].astype('category')
df_book_train_stock_X_noidx.head(2)

,stock_id,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,time_id_cat
0,1,5,0,1.000754,1.001542,1.000689,1.001607,1,25,25,100,5
1,1,5,1,1.000754,1.001673,1.000689,1.001739,26,60,25,100,5


In [61]:
%timeit df_book_train_stock_X_noidx.groupby('time_id_cat').mean()

132 ms ± 959 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [68]:
df_book_train_stock_X_noidx['stock_id_cat'] = df_book_train_stock_X_noidx['stock_id'].astype('category')
df_book_train_stock_X_idx_cat = df_book_train_stock_X_noidx.set_index(['stock_id_cat', 'time_id_cat'])
df_book_train_stock_X_idx_cat.head(2)

stock_id  time_id  seconds_in_bucket  bid_price1  \
stock_id_cat time_id_cat                                                     
1            5                   1        5                  0    1.000754   
             5                   1        5                  1    1.000754   

                          ask_price1  bid_price2  ask_price2  bid_size1  \
stock_id_cat time_id_cat                                                  
1            5              1.001542    1.000689    1.001607          1   
             5              1.001673    1.000689    1.001739         26   

                          ask_size1  bid_size2  ask_size2  
stock_id_cat time_id_cat                                   
1            5                   25         25        100  
             5                   60         25        100

In [70]:
%timeit df_book_train_stock_X_idx_cat.groupby('time_id_cat').mean()

149 ms ± 598 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
def make_features_stats(df_book, agg_type, cols):
    features_var1 = df_book.groupby(['stock_id', 'time_id'])[cols].agg(agg_type)
    #print(type(features_var1))
    if isinstance(features_var1, pd.Series):
        # .size yields a series not a df
        #features_var1.name = str(agg_type)
        features_var1 = pd.DataFrame(features_var1, columns=[agg_type])
        #pass
    else:
        features_var1_col_names = [f"{col}_{agg_type}" for col in cols]
        features_var1.columns = features_var1_col_names
    return features_var1


In [78]:
df_book_train_stock_X.groupby('time_id').mean()

,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
time_id,,,,,,,,,
5,297.786087,1.003313,1.003980,1.003201,1.004089,62.173913,110.916522,54.212174,60.386087
11,284.821622,1.001535,1.002020,1.001418,1.002124,75.372973,86.597297,80.445946,83.137838
16,309.141643,1.000827,1.001112,1.000709,1.001228,162.631728,151.328612,144.645892,167.492918
31,259.128655,0.997092,0.997864,0.996900,0.998013,99.853801,116.432749,100.514620,103.350877
62,309.669604,1.000377,1.001110,1.000259,1.001303,62.722467,84.299559,52.114537,77.938326
...,...,...,...,...,...,...,...,...,...
32751,281.788274,0.999821,1.000418,0.999663,1.000536,75.732899,59.938111,65.485342,95.029316
32753,294.089069,1.007074,1.008004,1.006915,1.008130,128.880567,185.609312,85.769231,167.580972
32758,295.258786,1.000424,1.001073,1.000284,1.001204,124.284345,71.469649,142.022364,88.827476


In [90]:
%timeit make_features_stats(df_book_train_stock_X, 'mean', df_book_train_stock_X.columns.values)

202 ms ± 3.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
